This notebook demonstrates how to traverse the WorldPeatland 
[SpatioTemporal Asset Catalog (STAC)](https://stacspec.org/) 
with [pystac](https://pystac.readthedocs.io/en/stable/).

Then open a Zarr dataset (STAC Asset) with xarray.

The WorldPeatland STAC has the following structure:

```mermaid
erDiagram
    "Root catalog" ||--|{ "Site catalog" : ""
    "Site catalog" ||--o{ "Collection (zarr)" : ""
    "Collection (zarr)" ||--|{ "Asset (zarr)" : ""
    "Site catalog" ||--o{ "Collection (cog)" : ""
    "Collection (cog)" ||--|| "Item" : ""
    "Item" ||--|{ "Asset" : ""
```

In [ ]:
CATALOG_URL = "https://s3.waw3-2.cloudferro.com/swift/v1/wpl-stac/stac/catalog.json"

In [ ]:
import pystac

In [ ]:
root: pystac.Catalog = pystac.read_file(CATALOG_URL)  # type: ignore
root

In [ ]:
# get the sub-catalog for Degero
catalog: pystac.Catalog = root.get_child("degero")  # type: ignore

The Albedo collection has 2 assets which are direct children of the collection.

These assets are zarr datacubes, one optimized for spatial queries and the other for temporal queries.

In [ ]:
# get the collection for Albedo
collection: pystac.Collection = catalog.get_child("albedo")  # type: ignore

In [ ]:
# how many assets are defined as direct children of the collection?
assets = collection.assets.values()
for asset in assets:
    display(asset)

Open one of the Zarr datasets (STAC Asset) with xarray.

In [ ]:
import rioxarray  # noqa: F401
import xarray as xr

In [ ]:
# this dataset is chunked for spatial reads
asset = collection.assets["albedo.xy.zarr"]

In [ ]:
# STAC extension "xarray Assets" provides keyword arguments
asset.ext.xarray.open_kwargs

In [ ]:
ds = xr.open_dataset(
    asset.href,
    **asset.ext.xarray.open_kwargs,  # type: ignore
)

In [ ]:
ds